# Compare prediction probabilities on validation

In [1]:
from pathlib import Path
import sys
sys.path.insert(0, Path(".").absolute().parent.as_posix())

In [2]:
import matplotlib.pylab as plt
%matplotlib inline
import seaborn as sns

import numpy as np
import pandas as pd

OUTPUT_PATH = Path(".").absolute().parent / "output" / "val_probas"

In [3]:
# Get true values
from PIL import Image
from common.dataset import FilesFromCsvDataset, TransformedDataset
from common.meta import get_metafeatures, get_imsize_and_targets


dataset = FilesFromCsvDataset("../output/filtered_val_dataset.csv")
dataset = TransformedDataset(dataset,
                             transforms=lambda x: (x, Image.open(x).size),
                             target_transforms=lambda l: l - 1)

df_imsize_targets = get_imsize_and_targets(dataset)
y_true = df_imsize_targets['target']

## Validation result from a single model

In [53]:
!ls {OUTPUT_PATH}

val_probas_inceptionresnetv2_350_resized_crop
val_probas_inceptionresnetv2_350_weighted_sampler2_resized_crop
val_probas_inceptionv4_350_cls_wts_adam_resized_crop
val_probas_inceptionv4_350_fc_random_resized_crop
val_probas_inceptionv4_350_resized_crop
val_probas_nasnetalarge_350_random_resized_crop
val_probas_nasnetalarge_350_resized_crop
val_probas_resnet152_350_adam_random_crop
val_probas_resnet152_350_resized_crop
val_probas_squeezenet_350


In [66]:
!ls {OUTPUT_PATH}/val_probas_resnet152_350_adam_random_crop/

20180522_1903  20180522_1905


In [95]:
prediction_file = OUTPUT_PATH / "val_probas_resnet152_350_adam_random_crop" / "20180522_1905" / "probas.csv"

In [96]:
df = pd.read_csv(prediction_file, index_col='id')

In [97]:
y_probas = df.groupby('id').mean()
y_probas = y_probas.loc[y_true.index, :]
y_pred = np.argmax(y_probas.values, axis=1)

In [99]:
from sklearn.metrics import accuracy_score

1.0 - accuracy_score(y_true, y_pred)

0.14560483230011123

### Results:

| Model | Val Error | Test 30% Error | Classes, Recall < 0.65 |
| --- | --- | --- | --- |
| inceptionresnetv2_350_resized_crop/20180428_1622 | 0.13829279923700522 | 0.15468 | (3, 0.45), (65, 0.5), (14, 0.24), (123, 0.61), (62, 0.3) |
| inceptionresnetv2_350_weighted_sampler2_resized_crop/20180501_1710 | 0.1702432045779685 | 0.17890 | (3, 0.58), (69, 0.61), (86, 0.58), (14, 0.42), (21, 0.57), (22, 0.41), (25, 0.58), (28, 0.59), (34, 0.58), (109, 0.64), (111, 0.56), (53, 0.63), (62, 0.5), (127, 0.62) |
| inceptionv4_350_cls_wts_adam_resized_crop/20180520_2026 | 0.14592274678111583 |  |(65, 0.58), (3, 0.54), (14, 0.28), (22, 0.63), (38, 0.64), (104, 0.62), (62, 0.42), (127, 0.65)  |
| inceptionv4_350_fc_random_resized_crop/20180520_2032 | 0.14353838817358133 | 0.16614 | (3, 0.37), (14, 0.36), (18, 0.58), (26, 0.63), (62, 0.3), (65, 0.54), (104, 0.60) |
| inceptionv4_350_resized_crop/20180428_1633 | 0.13718009855348912 | 0.15755 | (3, 0.46), (65, 0.44), (14, 0.3), (18, 0.56), (104, 0.65), (123, 0.53), (62, 0.3) |
| nasnetalarge_350_random_resized_crop/20180514_2202 | 0.1351136544269591 | 0.15026 | (3, 0.58), (65, 0.54), (14, 0.36), (26, 0.61), (38, 0.64), (62, 0.36) |
| nasnetalarge_350_random_resized_crop/20180514_2232 | 0.14147194404705132 | 0.15442 | (3, 0.48), (65, 0.5), (14, 0.46), (26, 0.59), (34, 0.65), (104, 0.62), (123, 0.49), (62, 0.32) |
| nasnetalarge_350_resized_crop/20180428_1654 | 0.1311397234144015 | 0.14635 |(3, 0.5625), (65, 0.4), (14, 0.38), (123, 0.55), (62, 0.3)|
| squeezenet_350/20180428_1447 | 0.24845016690510258 | | |
| resnet152_350_adam_random_crop/20180522_1903 | 0.15085042123668735 | | (3, 0.58), (65, 0.46), (14, 0.34), (18, 0.64), (123, 0.53), (62, 0.22) |
| resnet152_350_adam_random_crop/20180522_1905 | 0.14560483230011123 | | (3, 0.5), (65, 0.54), (14, 0.46), (18, 0.58), (96, 0.64), (104, 0.58), (49, 0.59), (123, 0.55), (62, 0.22) |



In [76]:
from sklearn.metrics import confusion_matrix, recall_score, precision_score, accuracy_score

cm = confusion_matrix(y_true, y_pred)
recall_per_class = recall_score(y_true, y_pred, average=None)

idx = np.arange(len(cm))
mcm = cm - cm[idx, idx] * np.eye(len(cm), dtype=np.int)
misclassifed = {}

for i in range(128):
    classes = np.where(mcm[i, :] >= 4)[0]
    values = mcm[i, classes]
    if len(classes) > 0:
        misclassifed[i] = {
            'recall': recall_per_class[i],
            'wrong_classes':[(c, v) for c, v in zip(classes, values)]
        }
        
hard_classes = [k for k in misclassifed if misclassifed[k]['recall'] < 0.65]
[(k, misclassifed[k]['recall']) for k in hard_classes]

[(3, 0.5),
 (65, 0.54),
 (14, 0.46),
 (18, 0.58),
 (96, 0.64),
 (104, 0.5833333333333334),
 (49, 0.5918367346938775),
 (123, 0.5510204081632653),
 (62, 0.22)]

All models recall per class

In [78]:
prediction_files = [
    OUTPUT_PATH / "val_probas_inceptionv4_350_resized_crop" / "20180428_1633" / "probas.csv",
    OUTPUT_PATH / "val_probas_inceptionv4_350_cls_wts_adam_resized_crop" / "20180520_2026" / "probas.csv",
    OUTPUT_PATH / "val_probas_inceptionv4_350_fc_random_resized_crop" / "20180520_2032" / "probas.csv",
    
    OUTPUT_PATH / "val_probas_resnet152_350_adam_random_crop" / "20180522_1903" / "probas.csv",
    OUTPUT_PATH / "val_probas_resnet152_350_adam_random_crop" / "20180522_1905" / "probas.csv",    
    
    OUTPUT_PATH / "val_probas_inceptionresnetv2_350_resized_crop" / "20180428_1622" / "probas.csv",
    OUTPUT_PATH / "val_probas_inceptionresnetv2_350_weighted_sampler2_resized_crop" / "20180501_1710" / "probas.csv",
    
    OUTPUT_PATH / "val_probas_nasnetalarge_350_resized_crop" / "20180428_1654" / "probas.csv",
    OUTPUT_PATH / "val_probas_nasnetalarge_350_random_resized_crop" / "20180514_2232" / "probas.csv",
    OUTPUT_PATH / "val_probas_nasnetalarge_350_random_resized_crop" / "20180514_2202" / "probas.csv",
]

In [149]:
names = [f.parent.parent.name[11:] + "/" + f.parent.name for f in prediction_files]

In [150]:
from sklearn.metrics import confusion_matrix, recall_score, precision_score, accuracy_score


recalls_df = pd.DataFrame(columns=names + ["mean", ], index=range(128))

for prediction_file, name in zip(prediction_files, names):

    df = pd.read_csv(prediction_file, index_col='id')

    y_probas = df.groupby('id').mean()
    y_probas = y_probas.loc[y_true.index, :]
    y_pred = np.argmax(y_probas.values, axis=1)

    recall_per_class = recall_score(y_true, y_pred, average=None)
    recalls_df.loc[:, name] = recall_per_class
    
    
dfs = [pd.read_csv(f, index_col='id') for f in prediction_files]
df_probas = pd.concat(dfs, axis=0)
y_probas = df_probas.groupby('id').mean()
y_probas = y_probas.loc[y_true.index, :]
y_pred = np.argmax(y_probas.values, axis=1)

recalls_df.loc[:, "mean"] = recall_score(y_true, y_pred, average=None)

In [153]:
recalls_df.head(20)

,inceptionv4_350_resized_crop/20180428_1633,inceptionv4_350_cls_wts_adam_resized_crop/20180520_2026,inceptionv4_350_fc_random_resized_crop/20180520_2032,resnet152_350_adam_random_crop/20180522_1903,resnet152_350_adam_random_crop/20180522_1905,inceptionresnetv2_350_resized_crop/20180428_1622,inceptionresnetv2_350_weighted_sampler2_resized_crop/20180501_1710,nasnetalarge_350_resized_crop/20180428_1654,nasnetalarge_350_random_resized_crop/20180514_2232,nasnetalarge_350_random_resized_crop/20180514_2202,mean
0,0.820000,0.860000,0.880000,0.860000,0.880000,0.860000,0.820000,0.860000,0.860000,0.880000,0.880000
1,0.780000,0.840000,0.840000,0.840000,0.840000,0.840000,0.760000,0.860000,0.840000,0.840000,0.840000
2,0.900000,0.900000,0.860000,0.880000,0.840000,0.900000,0.820000,0.900000,0.960000,0.940000,0.920000
3,0.458333,0.541667,0.375000,0.583333,0.500000,0.458333,0.583333,0.562500,0.479167,0.583333,0.541667
4,0.980000,0.980000,0.960000,0.960000,0.980000,0.960000,0.940000,0.960000,0.960000,0.960000,0.960000
5,0.914894,0.914894,0.872340,0.936170,0.808511,0.893617,0.872340,0.957447,0.978723,0.936170,0.914894
6,0.893617,0.829787,0.872340,0.872340,0.893617,0.893617,0.914894,0.914894,0.829787,0.872340,0.893617
7,0.920000,0.860000,0.900000,0.900000,0.900000,0.900000,0.800000,0.920000,0.840000,0.860000,0.920000
8,0.854167,0.833333,0.812500,0.833333,0.854167,0.875000,0.854167,0.916667,0.812500,0.833333,0.854167
9,0.957447,0.936170,0.957447,0.957447,0.914894,0.936170,0.914894,0.957447,0.978723,0.957447,0.957447


## Mean probability of multiple predictions

In [4]:
prediction_files = [
    OUTPUT_PATH / "val_probas_inceptionv4_350_resized_crop" / "20180428_1633" / "probas.csv",
    OUTPUT_PATH / "val_probas_inceptionv4_350_cls_wts_adam_resized_crop" / "20180520_2026" / "probas.csv",
    OUTPUT_PATH / "val_probas_inceptionv4_350_fc_random_resized_crop" / "20180520_2032" / "probas.csv",
    
    OUTPUT_PATH / "val_probas_resnet152_350_adam_random_crop" / "20180522_1903" / "probas.csv",
    OUTPUT_PATH / "val_probas_resnet152_350_adam_random_crop" / "20180522_1905" / "probas.csv",    
    
    OUTPUT_PATH / "val_probas_inceptionresnetv2_350_resized_crop" / "20180428_1622" / "probas.csv",
    OUTPUT_PATH / "val_probas_inceptionresnetv2_350_weighted_sampler2_resized_crop" / "20180501_1710" / "probas.csv",
    
    OUTPUT_PATH / "val_probas_nasnetalarge_350_resized_crop" / "20180428_1654" / "probas.csv",
    OUTPUT_PATH / "val_probas_nasnetalarge_350_random_resized_crop" / "20180514_2232" / "probas.csv",
    OUTPUT_PATH / "val_probas_nasnetalarge_350_random_resized_crop" / "20180514_2202" / "probas.csv",
]

In [5]:
dfs = [pd.read_csv(f, index_col='id') for f in prediction_files]
assert len(dfs) == len(prediction_files)

df_probas = pd.concat(dfs, axis=0)
y_probas = df_probas.groupby('id').mean()
y_probas = y_probas.loc[y_true.index, :]
y_pred = np.argmax(y_probas.values, axis=1)

In [6]:
from sklearn.metrics import accuracy_score

1.0 - accuracy_score(y_true, y_pred)

0.11858210141471948

Geometric mean probability

In [7]:
def gmean(arr):
    l = len(arr)
    prod = np.prod(arr)
    return np.power(prod, 1.0/l)

In [8]:
# y_probas_gmean = df_probas.groupby('id').agg(gmean)
# y_probas_gmean = y_probas_gmean.loc[y_true.index, :]
# y_pred_gmean = np.argmax(y_probas_gmean.values, axis=1)
# y_pred = np.argmax(y_probas.values, axis=1)

# 1.0 - accuracy_score(y_true, y_pred)

In [9]:
from sklearn.metrics import confusion_matrix, recall_score, precision_score, accuracy_score

cm = confusion_matrix(y_true, y_pred)
recall_per_class = recall_score(y_true, y_pred, average=None)

idx = np.arange(len(cm))
mcm = cm - cm[idx, idx] * np.eye(len(cm), dtype=np.int)
misclassifed = {}

for i in range(128):
    classes = np.where(mcm[i, :] >= 4)[0]
    values = mcm[i, classes]
    if len(classes) > 0:
        misclassifed[i] = {
            'recall': recall_per_class[i],
            'wrong_classes':[(c, v) for c, v in zip(classes, values)]
        }
        
hard_classes = [k for k in misclassifed if misclassifed[k]['recall'] < 0.65]
[(k, misclassifed[k]['recall']) for k in hard_classes]

[(3, 0.5416666666666666),
 (65, 0.56),
 (14, 0.38),
 (123, 0.6122448979591837),
 (62, 0.3)]

#### Test 30% score:

In [10]:
OUTPUT_PATH = Path(".").absolute().parent / "output" / "test_probas"

test_prediction_files = [
    
    OUTPUT_PATH / "test_probas_inceptionv4_350_resized_crop" / "20180429_1303" / "probas.csv",
    OUTPUT_PATH / "test_probas_inceptionv4_350_cls_wts_adam_resized_crop" / "20180520_2151" / "probas.csv",
    OUTPUT_PATH / "test_probas_inceptionv4_350_fc_random_resized_crop" / "20180520_2151" / "probas.csv",
    
    OUTPUT_PATH / "test_probas_resnet152_350_adam_random_crop" / "20180522_2014" / "probas.csv",
    OUTPUT_PATH / "test_probas_resnet152_350_adam_random_crop" / "20180522_2039" / "probas.csv",
        
    OUTPUT_PATH / "test_probas_inceptionresnetv2_350_resized_crop" / "20180429_1242" / "probas.csv",
    OUTPUT_PATH / "test_probas_inceptionresnetv2_350_weighted_sampler2_resized_crop" / "20180501_1725" / "probas.csv",
    
    OUTPUT_PATH / "test_probas_nasnetalarge_350_resized_crop" / "20180429_1406" / "probas.csv",
    OUTPUT_PATH / "test_probas_nasnetalarge_350_random_resized_crop" / "20180514_1034" / "probas.csv",    
    OUTPUT_PATH / "test_probas_nasnetalarge_350_random_resized_crop" / "20180514_0821" / "probas.csv",
]

test_dfs = [pd.read_csv(f, index_col='id') for f in test_prediction_files]

sample_predicitions_df = pd.read_csv("/home/fast_storage/imaterialist-challenge-furniture-2018/sample_submission_randomlabel.csv", index_col='id')

df_test_probas = pd.concat(test_dfs, axis=0)
y_test_probas = df_test_probas.groupby('id').sum()
y_test_pred = np.argmax(y_test_probas.values, axis=1)

df = sample_predicitions_df.copy()
df.loc[y_test_probas.index, 'predicted'] = y_test_pred + 1
df.to_csv("mean_proba_10models.csv")

In [11]:
!cd ../ && python3 utils/update_test_predictions.py notebooks/mean_proba_10models.csv notebooks/test_with_labels.csv notebooks/

In [12]:
!head -10 mean_proba_10models.csv

id,predicted
1,12
2,71
3,91
4,54
5,126
6,76
7,94
8,8
9,127


Test 30% score: 0.13385

### Predictions weighted by recall per class

In [120]:
recalls_df.head(10)

,inceptionv4_350_resized_crop/20180428_1633,inceptionv4_350_cls_wts_adam_resized_crop/20180520_2026,inceptionv4_350_fc_random_resized_crop/20180520_2032,resnet152_350_adam_random_crop/20180522_1903,resnet152_350_adam_random_crop/20180522_1905,inceptionresnetv2_350_resized_crop/20180428_1622,inceptionresnetv2_350_weighted_sampler2_resized_crop/20180501_1710,nasnetalarge_350_resized_crop/20180428_1654,nasnetalarge_350_random_resized_crop/20180514_2232,nasnetalarge_350_random_resized_crop/20180514_2202
0,0.820000,0.860000,0.880000,0.860000,0.880000,0.860000,0.820000,0.860000,0.860000,0.880000
1,0.780000,0.840000,0.840000,0.840000,0.840000,0.840000,0.760000,0.860000,0.840000,0.840000
2,0.900000,0.900000,0.860000,0.880000,0.840000,0.900000,0.820000,0.900000,0.960000,0.940000
3,0.458333,0.541667,0.375000,0.583333,0.500000,0.458333,0.583333,0.562500,0.479167,0.583333
4,0.980000,0.980000,0.960000,0.960000,0.980000,0.960000,0.940000,0.960000,0.960000,0.960000
5,0.914894,0.914894,0.872340,0.936170,0.808511,0.893617,0.872340,0.957447,0.978723,0.936170
6,0.893617,0.829787,0.872340,0.872340,0.893617,0.893617,0.914894,0.914894,0.829787,0.872340
7,0.920000,0.860000,0.900000,0.900000,0.900000,0.900000,0.800000,0.920000,0.840000,0.860000
8,0.854167,0.833333,0.812500,0.833333,0.854167,0.875000,0.854167,0.916667,0.812500,0.833333
9,0.957447,0.936170,0.957447,0.957447,0.914894,0.936170,0.914894,0.957447,0.978723,0.957447


In [241]:
prediction_files = [
    OUTPUT_PATH / "val_probas_inceptionv4_350_resized_crop" / "20180428_1633" / "probas.csv",
    OUTPUT_PATH / "val_probas_inceptionv4_350_cls_wts_adam_resized_crop" / "20180520_2026" / "probas.csv",
    OUTPUT_PATH / "val_probas_inceptionv4_350_fc_random_resized_crop" / "20180520_2032" / "probas.csv",
    
    OUTPUT_PATH / "val_probas_resnet152_350_adam_random_crop" / "20180522_1903" / "probas.csv",
    OUTPUT_PATH / "val_probas_resnet152_350_adam_random_crop" / "20180522_1905" / "probas.csv",    
    
    OUTPUT_PATH / "val_probas_inceptionresnetv2_350_resized_crop" / "20180428_1622" / "probas.csv",
    OUTPUT_PATH / "val_probas_inceptionresnetv2_350_weighted_sampler2_resized_crop" / "20180501_1710" / "probas.csv",
    
    OUTPUT_PATH / "val_probas_nasnetalarge_350_resized_crop" / "20180428_1654" / "probas.csv",
    OUTPUT_PATH / "val_probas_nasnetalarge_350_random_resized_crop" / "20180514_2232" / "probas.csv",
    OUTPUT_PATH / "val_probas_nasnetalarge_350_random_resized_crop" / "20180514_2202" / "probas.csv",
]

dfs = [pd.read_csv(f, index_col='id') for f in prediction_files]

In [276]:
w = 0.4
0.5 * np.tanh((w - 0.35)*5) + 0.5

0.5

In [283]:
names = recalls_df.columns

def weight_preproc(w):
    return 0.5 * np.tanh((w - 0.35)*5) + 0.5


weighted_dfs = [df.mul(weight_preproc(recalls_df.loc[:, name].values), axis='columns') for name, df in zip(names, dfs)]

df_probas = pd.concat(weighted_dfs, axis=0)
y_probas = df_probas.groupby('id').sum()
y_probas = y_probas.loc[y_true.index, :]
y_pred = np.argmax(y_probas.values, axis=1)

1.0 - accuracy_score(y_true.values, y_pred)

0.11762835797170557

In [284]:
from sklearn.metrics import confusion_matrix, recall_score, precision_score, accuracy_score

cm = confusion_matrix(y_true, y_pred)
recall_per_class = recall_score(y_true, y_pred, average=None)

idx = np.arange(len(cm))
mcm = cm - cm[idx, idx] * np.eye(len(cm), dtype=np.int)
misclassifed = {}

for i in range(128):
    classes = np.where(mcm[i, :] >= 4)[0]
    values = mcm[i, classes]
    if len(classes) > 0:
        misclassifed[i] = {
            'recall': recall_per_class[i],
            'wrong_classes':[(c, v) for c, v in zip(classes, values)]
        }
        
hard_classes = [k for k in misclassifed if misclassifed[k]['recall'] < 0.65]
[(k, misclassifed[k]['recall']) for k in hard_classes]

[(3, 0.5625),
 (65, 0.56),
 (14, 0.34),
 (104, 0.6458333333333334),
 (123, 0.6122448979591837),
 (62, 0.26)]

In [285]:
recalls_df.loc[62, :]

inceptionv4_350_resized_crop/20180428_1633                            0.30
inceptionv4_350_cls_wts_adam_resized_crop/20180520_2026               0.42
inceptionv4_350_fc_random_resized_crop/20180520_2032                  0.30
resnet152_350_adam_random_crop/20180522_1903                          0.22
resnet152_350_adam_random_crop/20180522_1905                          0.22
inceptionresnetv2_350_resized_crop/20180428_1622                      0.30
inceptionresnetv2_350_weighted_sampler2_resized_crop/20180501_1710    0.50
nasnetalarge_350_resized_crop/20180428_1654                           0.30
nasnetalarge_350_random_resized_crop/20180514_2232                    0.32
nasnetalarge_350_random_resized_crop/20180514_2202                    0.36
mean                                                                  0.30
Name: 62, dtype: float64

Mean probas predictions recall < 0.65:
```
[(3, 0.5416666666666666),
 (65, 0.56),
 (14, 0.38),
 (123, 0.6122448979591837),
 (62, 0.3)]
```

### Random search of weights

In [89]:
from sklearn.metrics import f1_score

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [14]:
from sklearn.model_selection import StratifiedKFold

splt = StratifiedKFold(n_splits=5, random_state=17)
dfs = [pd.read_csv(f, index_col='id') for f in prediction_files]

In [106]:
def score(params):
    weights = [params["w_{}".format(i)] for i in range(len(dfs))]
    
#     print("Trial: weights: {}".format(weights), end=" -> ")
    weighted_dfs = [w * df for w, df in zip(weights, dfs)]
    df_probas = pd.concat(weighted_dfs, axis=0)
    y_probas = df_probas.groupby('id').sum()
    y_probas = y_probas.loc[y_true.index, :]
    y_pred = np.argmax(y_probas.values, axis=1)
    
    res_cv = []
    res_test = []
    for train_index, test_index in splt.split(y_true.values[:, None], y_true.values):        
        res_cv.append(1.0 - f1_score(y_true.values[train_index], y_pred[train_index], average='micro'))
        res_test.append(1.0 - f1_score(y_true.values[test_index], y_pred[test_index], average='micro'))        
    
    res = np.mean(res_cv)
    res_test = np.mean(res_test)
#     print("Score: cv={} | test={}".format(res, res_test))
    return {'loss': res, 'status': STATUS_OK, 'val_score': res_test}
    
    
def get_uniform(name):
    return hp.quniform(name, 0.1, 0.9, 0.001)
    
    
def optimize(trials, max_evals):
    params = {}
    for i in range(len(prediction_files)):
        n = "w_{}".format(i)
        params[n] = get_uniform(n)   
    best = fmin(score, params, algo=tpe.suggest, trials=trials, max_evals=max_evals)    
    return best


trials = Trials()
best_weights = optimize(trials, max_evals=2500)
best_weights

{'w_0': 0.794,
 'w_1': 0.258,
 'w_2': 0.577,
 'w_3': 0.628,
 'w_4': 0.11,
 'w_5': 0.8310000000000001,
 'w_6': 0.495,
 'w_7': 0.114,
 'w_8': 0.249,
 'w_9': 0.863}

In [107]:
best_weights, trials.best_trial

({'w_0': 0.794,
  'w_1': 0.258,
  'w_2': 0.577,
  'w_3': 0.628,
  'w_4': 0.11,
  'w_5': 0.8310000000000001,
  'w_6': 0.495,
  'w_7': 0.114,
  'w_8': 0.249,
  'w_9': 0.863},
 {'book_time': datetime.datetime(2018, 5, 23, 0, 31, 21, 468000),
  'exp_key': None,
  'misc': {'cmd': ('domain_attachment', 'FMinIter_Domain'),
   'idxs': {'w_0': [1493],
    'w_1': [1493],
    'w_2': [1493],
    'w_3': [1493],
    'w_4': [1493],
    'w_5': [1493],
    'w_6': [1493],
    'w_7': [1493],
    'w_8': [1493],
    'w_9': [1493]},
   'tid': 1493,
   'vals': {'w_0': [0.794],
    'w_1': [0.258],
    'w_2': [0.577],
    'w_3': [0.628],
    'w_4': [0.11],
    'w_5': [0.8310000000000001],
    'w_6': [0.495],
    'w_7': [0.114],
    'w_8': [0.249],
    'w_9': [0.863]},
   'workdir': None},
  'owner': None,
  'refresh_time': datetime.datetime(2018, 5, 23, 0, 31, 21, 574000),
  'result': {'loss': 0.11603594619976283,
   'status': 'ok',
   'val_score': 0.1159940337433106},
  'spec': None,
  'state': 2,
  'tid': 14

In [108]:
weights = [best_weights["w_{}".format(i)] for i in range(len(dfs))]

weighted_dfs = [w * df for w, df in zip(weights, dfs)]
df_probas = pd.concat(weighted_dfs, axis=0)
y_probas = df_probas.groupby('id').sum()
y_probas = y_probas.loc[y_true.index, :]
y_pred = np.argmax(y_probas.values, axis=1)

1.0 - accuracy_score(y_true.values, y_pred)

0.11603878556668257

In [291]:
weights = [best_weights["w_{}".format(i)] for i in range(len(dfs))]

weighted_dfs = [w * df for w, df in zip(weights, dfs)]
df_probas = pd.concat(weighted_dfs, axis=0)
y_probas = df_probas.groupby('id').sum()
y_probas = y_probas.loc[y_true.index, :]
y_pred = np.argmax(y_probas.values, axis=1)

1.0 - accuracy_score(y_true.values, y_pred)

0.11556191384517567

### Compute test probabilities

In [94]:
# best_weights = {
#     'w_0': 0.8654605129896079,
#     'w_1': 0.5321224537280229,
#     'w_2': 0.1886420238665844,
#     'w_3': 0.7444695911930883,
#     'w_4': 0.24655462551634644,
#     'w_5': 0.5547166610070141,
#     'w_6': 0.4298804200059071,
#     'w_7': 0.3334794430497531
# }

# Model A
# best_weights = {
#     'w_0': 0.6289182255335299,
#     'w_1': 0.8198013546612819,
#     'w_2': 0.8206407193353897,
#     'w_3': 0.8075748855704723,
#     'w_4': 0.47081185000824,
#     'w_5': 0.34978984178522243,
#     'w_6': 0.15526962510582593,
#     'w_7': 0.814422311728778,
#     'w_8': 0.809143873522562,
#     'w_9': 0.4938244150400559
# }


# # Model B
# best_weights = {
#     'w_0': 0.7151023288481337,
#     'w_1': 0.402703332723889,
#     'w_2': 0.6815185881586352,
#     'w_3': 0.5457820237214128,
#     'w_4': 0.194473834092752,
#     'w_5': 0.4864916299616243,
#     'w_6': 0.13515340243314064,
#     'w_7': 0.48117926414156037,
#     'w_8': 0.8231804838744279,
#     'w_9': 0.3622520857862331
# }

# Model C
# best_weights = {
#   'w_0': 0.9,
#   'w_1': 0.28,
#   'w_2': 0.62,
#   'w_3': 0.4,
#   'w_4': 0.3,
#   'w_5': 0.70,
#   'w_6': 0.18,
#   'w_7': 0.22,
#   'w_8': 0.74,
#   'w_9': 0.26
# }

# Model D
best_weights = {
    'w_0': 0.88,
    'w_1': 0.44,
    'w_2': 0.6,
    'w_3': 0.54,
    'w_4': 0.34,
    'w_5': 0.6,
    'w_6': 0.2,
    'w_7': 0.22,
    'w_8': 0.84,
    'w_9': 0.48
}

In [95]:
OUTPUT_PATH = Path(".").absolute().parent / "output" / "test_probas"

In [96]:
!ls {OUTPUT_PATH}/test_probas_resnet152_350_adam_random_crop/

20180522_2014  20180522_2039


In [97]:
# OUTPUT_PATH = Path(".").absolute().parent / "output" / "val_probas"

# !tail -20 {OUTPUT_PATH}/"val_probas_resnet152_350_adam_random_crop"/"20180522_1905"/"val_probas_resnet152_350_adam_random_crop.py"

In [98]:
# OUTPUT_PATH = Path(".").absolute().parent / "output" / "test_probas"

# !tail -20 {OUTPUT_PATH}/"test_probas_resnet152_350_adam_random_crop"/"20180522_2039"/"test_probas_resnet152_350_adam_random_crop.py"

In [99]:
test_prediction_files = [
    
    OUTPUT_PATH / "test_probas_inceptionv4_350_resized_crop" / "20180429_1303" / "probas.csv",
    OUTPUT_PATH / "test_probas_inceptionv4_350_cls_wts_adam_resized_crop" / "20180520_2151" / "probas.csv",
    OUTPUT_PATH / "test_probas_inceptionv4_350_fc_random_resized_crop" / "20180520_2151" / "probas.csv",
    
    OUTPUT_PATH / "test_probas_resnet152_350_adam_random_crop" / "20180522_2014" / "probas.csv",
    OUTPUT_PATH / "test_probas_resnet152_350_adam_random_crop" / "20180522_2039" / "probas.csv",

    OUTPUT_PATH / "test_probas_inceptionresnetv2_350_resized_crop" / "20180429_1242" / "probas.csv",
    OUTPUT_PATH / "test_probas_inceptionresnetv2_350_weighted_sampler2_resized_crop" / "20180501_1725" / "probas.csv",
    
    OUTPUT_PATH / "test_probas_nasnetalarge_350_resized_crop" / "20180429_1406" / "probas.csv",
    OUTPUT_PATH / "test_probas_nasnetalarge_350_random_resized_crop" / "20180514_1034" / "probas.csv",    
    OUTPUT_PATH / "test_probas_nasnetalarge_350_random_resized_crop" / "20180514_0821" / "probas.csv",    
]

In [100]:
weights = [best_weights["w_{}".format(i)] for i in range(len(test_prediction_files))]
test_dfs = [w * pd.read_csv(f, index_col='id') for w, f in zip(weights, test_prediction_files)]

In [101]:
sample_predicitions_df = pd.read_csv("/home/fast_storage/imaterialist-challenge-furniture-2018/sample_submission_randomlabel.csv", index_col='id')

In [102]:
sample_predicitions_df.head()

,predicted
id,
1,57
2,74
3,52
4,54
5,39


In [103]:
df_test_probas = pd.concat(test_dfs, axis=0)
y_test_probas = df_test_probas.groupby('id').sum()
y_test_pred = np.argmax(y_test_probas.values, axis=1)

df = sample_predicitions_df.copy()
df.loc[y_test_probas.index, 'predicted'] = y_test_pred + 1
df.to_csv("weighted_proba_10models_d.csv")

In [104]:
!cd ../ && python3 utils/update_test_predictions.py notebooks/weighted_proba_10models_d.csv notebooks/test_with_labels.csv notebooks/

In [105]:
!head -10 fixed_weighted_proba_10models_d.csv

id,predicted
1,12
2,71
3,91
4,54
5,126
6,76
7,94
8,8
9,127


```
id,predicted
1,12
2,71
3,91
4,54
5,126
6,76
7,94
8,8
9,127
```